# pandas数据结构

pandas提供了两种数据结构：Series和DataFrame。

首先导入pandas相关的模块。

In [13]:
import pandas as pd
import numpy as np

## Series

### 概述

Series是一个包含一列数据的一维数组形式的对象，其中每一项数据都关联了一个数据标签作为索引，Series的默认索引是以0为起始的递增数列。

一个最简单的Series如下所示。

In [2]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

可以通过Series对象的index和values属性返回该Series对象的索引和值列表。

In [3]:
# obj的索引
print(obj.index)
# obj的值
print(obj.values)

RangeIndex(start=0, stop=4, step=1)
[ 4  7 -5  3]


### 在创建Series时提供索引列表

可以在创建Series对象时指定index属性值，为Series对象提供索引。

In [4]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

### 通过索引返回Series中元素引用(\[\]操作符的索引与赋值操作)

与数组操作类似，可以通过Series的索引选择并返回一个或多个值，或对一个或多个值进行赋值操作。

**索引操作**

\[\]支持对给定的单个索引元素返回单个元素的功能，也支持对给定的索引列表返回多个元素的功能，还支持对给定的筛选条件返回满足条件的所有元素的筛选功能。

通过\[\]操作符查询返回Series中的多个值时，传递给\[\]的参数是一个索引列表，并且返回的是一个当前Series的子集Series对象。

对只有单个索引元素的列表的\[\]操作，则返回包含单个元素的Series对象，这与传递给\[\]单个索引元素是有区别的。

In [5]:
# 返回单个值
obj2['a']

-5

In [6]:
# 返回多个值
obj2[['c', 'a', 'd']]

c    3
a   -5
d    4
dtype: int64

In [9]:
# 返回仅包含一个元素的Series对象
obj2[['a']]

a   -5
dtype: int64

In [10]:
# 返回Sereis对象中元素值大于0的元素
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

**赋值操作**

\[\]操作符支持对Series对象中的单个或多个元素同时赋值的功能，这种赋值操作将更改原Series元素的值。

同时也支持Series对象上直接的运算操作。

In [7]:
# 单个元素赋值
obj2['d'] = 6
obj2['d']

6

In [11]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [14]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

### Series对象支持字典类型对象的操作

Series也可以看做是固定长度的有序dict对象，其中键为索引，值为元素值。

因此Series对象也支持大部分dict对象的操作，如in操作符。同时Series也支持通过dict对象创建Series对象，dict对象的键作为Series对象索引，而值作为Sereis的元素值。

但Series不支持如dict形式的for循环遍历操作。

In [15]:
'b' in obj2

True

In [18]:
# 通过dict对象创建Series对象
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

通过在创建Series时提供index属性值，可以对Series对象根据索引值进行排序。对于给定的索引列表中缺省的索引值，Series对象中该索引对应的值为NaN。

In [19]:
# 创建Series对索引进行排序
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

### 处理空值NaN

Series支持对空值或非空值的筛选功能。

In [20]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [21]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [22]:
obj4[pd.notnull(obj4)]

Ohio      35000.0
Oregon    16000.0
Texas     71000.0
dtype: float64

### Series基于索引连接与代数运算

Series支持基于索引连接后进行代数运算的功能。

In [24]:
# 按索引将对元素值求和
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

### Series命名

Series对象与其索引都有一个name属性，用于对Series对象与索引命名

In [26]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

### Series中的索引赋值

Series中的索引可以通过赋值操作直接修改。

In [27]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

## DataFrame

### 概述

DataFrame可以看做一个或多个具有相同索引的Series对象排列组合构成的一个有序dict对象，其中DataFrame的每一列即为一个Series对象，而每一列对应的column属性即为键值。

但其底层是以一个或多个二维块形式存储的。

### 创建DataFrame

创建DataFrame的方式有多种：

可以通过一个dict来创建DataFrame对象，dict中的各个键对应的值均为等长的list对象。每一键即为DataFrame的column属性值，而每一个值即为DataFrame的每一列的值，索引默认为从0开始的底层序列。

In [29]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)
frame

,pop,state,year
0,1.5,Ohio,2000
1,1.7,Ohio,2001
2,3.6,Ohio,2002
3,2.4,Nevada,2001
4,2.9,Nevada,2002
5,3.2,Nevada,2003


同样的，你也可以指定DataFrame的columns与index属性，无对应值的columns与index列表元素对应的值为NaN。

In [31]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                            index=['one', 'two', 'three', 'four','five', 'six'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


除此之外，还可以通过一个嵌套的dict对象来创建DataFrame对象，其中，外层dict的键作为DataFrame的columns，而内层dict的键作为index，内层dict的值作为元素值。

In [41]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}
frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


类似的，还可以通过一个值为Series对象的dict对象创建DataFrame，dict对象的键作为DataFrame的columns，dict对象的值Series作为DataFrame的每一列。

In [44]:
pdata = {'Ohio': frame3['Ohio'][:-1],
         'Nevada': frame3['Nevada'][:2]}
pd.DataFrame(pdata)

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7


DataFrame构造器能够接受的所有输入数据类型如下表：

|类型|备注|
|:-|:-|
|二维ndarray类型|数据矩阵|
|value类型为array,list或tuple的dict|dict的每一个value作为DataFrame的列，每一个序列类型必须长度相同|
|numpy structured/record array|类似于array类型的dict|
|Series类型的dict|dict的每一个value作为DataFrame的列，每一个Series的index|
|嵌套dict||
|dict或Series类型的list||
|list或tuple类型的list||
|DataFrame||
|numpy MaskedArray||

### 返回DataFrame的行与列（\[\]操作符的索引与赋值操作）

DataFrame可以根据列的column属性返回一列或多列，也可以根据行的index属性返回一行或多行。并且可以对

**索引操作**

In [32]:
# 返回DataFrame的一列
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [33]:
# 返回DataFrame的一行
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

**赋值操作**

DataFrame支持将一个标量值或一个数组的各个元素依次赋值给其中一列的所有元素，也可以直接将一个Series对象按照其对应的索引值赋值给DataFrame中一列对应的元素。

In [35]:
# 为DataFrame的一列赋值一个标量值
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,3.2,16.5


In [37]:
# 为DataFrame的一列中每个元素依次赋值一个数组元素值
frame2['debt'] = np.arange(6.0)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


In [38]:
# 根据对应的索引值将一个Series对象赋值给DataFrame的一列
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


### del

del可以用于删除DataFrame指定的一列。

In [39]:
# 为DataFrame添加一列
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


In [40]:
# 删除指定的一列
del frame2['eastern']
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

### DataFrame支持的其他操作

In [42]:
# DataFrame转置运算
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


In [45]:
pd.DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


## Index对象

pandas中，Index类型对象用于表示Series与DataFrame的index与columns属性，在构造Series和DataFrame时，传给index或columns属性的列表与序列将会被转换为Index类型对象。

In [47]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

Index类型对象与tuple类型类似，也支持slice操作。Index对象是不可变对象，不能对其进行赋值操作，Index对象的不可变性使得其在传递中更加安全。

In [48]:
index[1:]

Index(['b', 'c'], dtype='object')

In [50]:
# wrong
# index[1] = 'd'

In [52]:
labels = pd.Index(np.arange(3))
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2.index is labels

True

Index对象同样也支持与不可变set类型类似的操作。

但与set不同的是，Index对象允许包含重复的值。

In [55]:
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [53]:
'Ohio' in frame3.columns

True

In [54]:
2003 in frame3.index

False

In [56]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

Index对象支持的方法如下表所示：

|方法|备注|
|-|-|
|append||
|difference||
|intersection||
|union||
|isin||
|delete||
|drop||
|insert||
|is_monotonic||
|is_unique||
|unique||
